In [1]:
import re
import os
import csv
import json
import requests
from urllib import parse
from bs4 import BeautifulSoup
from locals.output_data import exportData
from locals.date_scrape import getDate, lastStamp, searchDate
from locals.automate import By, Keys, EC, Action, wait_for, genBrowser, driverWait

In [2]:
# setting up user agent

header_1 = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0'
}

header_2 = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

header_3 = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:109.0) Gecko/20100101 Firefox/114.0'
}

header_4 = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0'
}

header_5 = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

In [3]:
def get_data(html, data_list, base_url):
    soup = BeautifulSoup(html, 'lxml')
    page_content = soup.select_one('section.content')

    category = page_content.select_one('div.head span[itemprop="name"]').string
    topic_name = page_content.select_one('div.showthread__title h1').get_text(strip=True)

    posts = page_content.select('div#posts div.showthread')

    for post in posts:

        # fixing the time scrape selector
        date_tag = soup.select('div.threadpost-header__controls li.threadpost-header__controllink--nolink')
        date_text = date_tag.get_text(strip=True)
        pattern = r"(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}\,\s20\d{2}"

        match = re.search(pattern, date_text)

        if match:
            date_string = match.group()
        else:
            print('date find error!')
            date_string = None

        has_files = post.select_one('div.threadpost-content div.threadpost-content__attachments')
        if has_files:
            post_details_tag = post.select_one('div.threadpost-header div.threadpost-header__controls a[title="Post Permalink"]')
            post_url = parse.urljoin(base_url, post_details_tag['href'])
            post_count = post_details_tag['data-postnum']

            info_text = file.select_one('span.info').get_text(strip=True)
            download_text = re.search(r"\b\|[0-9\,]+\sdownloads\b", info_text).group()
            downloads = download_text.replace('|', '').replace('downloads', '').replace(',', '').strip()
            
            attach_files = has_files.select('div.attachinfo')

            for file in attach_files:
                file_url = file.a['href']
                file_name = file.a.string

                data_dict = {
                    "file_name": file_name,
                    "download_url": file_url,
                    "downloads": downloads,
                    "date": date_string,
                    "topic_name": topic_name,
                    "category": category,
                    "post_url": post_url,
                    "post#": post_count
                }

                data_list.append(data_dict)
                # print(data_dict)     
        else:
            continue

In [6]:
# data to be collected
def main():
    error_url_list = []
    data_list = []

    session = requests.Session()
    pages = [page for page in range(1, 85)]

    for page in pages:
        print(f"crawling page: {page}")
        base_url = 'https://www.forexfactory.com'
        extend_url = f'/thread/594890-indicator-bank?page={page}'
        target_url = parse.urljoin(base_url, extend_url)

        res = session.get(target_url, headers=header_1)

        if res.status_code == 200:
            html = res.text
            
            try:
                get_data(html, data_list, base_url)
            except:
                break
        else:
            print(f'Bad Response- {res.status_code}')
            error_url_list.append(target_url)
    
    return (error_url_list, data_list)

In [7]:
# run the crawler

error_list, data_list = main()
exportData('forex_factory', data_list)
# print(error_list)

crawling page: 1
crawling page: 2
crawling page: 3
crawling page: 4
crawling page: 5
crawling page: 6
crawling page: 7
crawling page: 8
crawling page: 9
crawling page: 10
crawling page: 11
crawling page: 12
crawling page: 13
crawling page: 14
crawling page: 15
crawling page: 16
crawling page: 17
crawling page: 18
crawling page: 19
crawling page: 20
crawling page: 21
crawling page: 22
crawling page: 23
crawling page: 24
crawling page: 25
crawling page: 26
crawling page: 27
crawling page: 28
crawling page: 29
crawling page: 30
crawling page: 31
crawling page: 32
crawling page: 33
crawling page: 34
crawling page: 35
crawling page: 36
crawling page: 37
crawling page: 38
crawling page: 39
crawling page: 40
crawling page: 41
crawling page: 42
crawling page: 43
crawling page: 44
crawling page: 45
crawling page: 46
crawling page: 47
crawling page: 48
crawling page: 49
crawling page: 50
crawling page: 51
crawling page: 52
crawling page: 53
crawling page: 54
crawling page: 55
crawling page: 56
c